In [ ]:
import pandas as pd
from pyarrow.parquet import ParquetFile
import dask.dataframe as dd
import os
import xarray as xr
import ujson
import pprint

#%matplotlib inline

In [ ]:
# These fs options don't work for http... beware!
so = dict(mode="rb", anon=True, default_fill_cache=False, default_cache_type="first")

In [ ]:
def gen_json(u, fs, outf=None):
    with fs.open(u, **so) as infile:
        h5chunks = SingleHdf5ToZarr(infile, u, inline_threshold=300)
        p = u.split("/")
        date = p[3]
        fname = p[5]
        if outf:
            # outf = f'{json_dir}{date}.{fname}.json'
            with open(outf, "wb") as f:
                f.write(ujson.dumps(h5chunks.translate()).encode())
        else:
            return h5chunks.translate()

In [ ]:
# dir_files = [os.path.join("../short_range_18files", files) for files in os.listdir("../short_range_18files")]
# dir_files = [os.path.join("short_range_2files", files) for files in os.listdir("short_range_2files")]
# print(dir_files)
dir_files = [
    "nwm.t00z.short_range.channel_rt.f001.conus.nc",
    "nwm.t00z.short_range.channel_rt.f002.conus.nc",
    "nwm.t00z.short_range.channel_rt.f003.conus.nc",
    "nwm.t00z.short_range.channel_rt.f004.conus.nc",
    "nwm.t00z.short_range.channel_rt.f005.conus.nc",
    "nwm.t00z.short_range.channel_rt.f006.conus.nc",
    "nwm.t00z.short_range.channel_rt.f007.conus.nc",
    "nwm.t00z.short_range.channel_rt.f008.conus.nc",
    "nwm.t00z.short_range.channel_rt.f009.conus.nc",
    "nwm.t00z.short_range.channel_rt.f010.conus.nc",
    "nwm.t00z.short_range.channel_rt.f011.conus.nc",
    "nwm.t00z.short_range.channel_rt.f012.conus.nc",
    "nwm.t00z.short_range.channel_rt.f013.conus.nc",
    "nwm.t00z.short_range.channel_rt.f014.conus.nc",
    "nwm.t00z.short_range.channel_rt.f015.conus.nc",
    "nwm.t00z.short_range.channel_rt.f016.conus.nc",
    "nwm.t00z.short_range.channel_rt.f017.conus.nc",
    "nwm.t00z.short_range.channel_rt.f018.conus.nc",
]

In [ ]:
import fsspec
import xarray as xr
from kerchunk.hdf import SingleHdf5ToZarr

fs = fsspec.filesystem("gcs", anon=True)

# https://storage.googleapis.com/national-water-model/nwm.20220911/short_range/nwm.t00z.short_range.channel_rt.f001.conus.nc
# gcs_url = "gcs://national-water-model/nwm.20220911/short_range/nwm.t00z.short_range.channel_rt.f001.conus.nc"
gcs_url = "gcs://national-water-model/nwm.20220911/short_range/"

sr_h5 = []
for f in dir_files:
    print(f)
    sr_h5.append(gen_json(gcs_url + f, fs))

In [ ]:
%%time
fds = []
for xj in sr_h5:
    backend_args = {
        "consolidated": False,
        "storage_options": {
            "fo": xj,
            # Adding these options returns a properly dimensioned but otherwise null dataframe
            # "remote_protocol": "https",
            # "remote_options": {'anon':True}
        },
    }
    fds.append(
        xr.open_dataset(
            "reference://",
            engine="zarr",
            mask_and_scale=False,
            backend_kwargs=backend_args,
        )
    )

In [ ]:
%%time
ds = xr.concat(fds, dim="time")
ds

In [ ]:
df = ds["streamflow"].to_dataframe()

In [ ]:
# df = df.streamflow
df

In [ ]:
%%time
# df = pd.Series.to_frame(df)

df.to_parquet(
    "../data/parquet_all_feature_ids.gzip", engine="pyarrow", compression="gzip"
)

In [ ]:
ParquetFile("../data/parquet_all_feature_ids.gzip").metadata  #  num_columns: 3

In [ ]:
%%time
data = dd.read_parquet(
    "../data/parquet_all_feature_ids.gzip", storage_options={"anon": True}
)
data
result = data.compute()

In [ ]:
# result = result.loc[:, [101]]
result = result.loc[:, 100:1032]
# result= result.loc[:, :, 1000:11000]
# result= result.loc[:, :, 10000:110000]
result
r_xa = result.to_xarray()
r_xa
r_xa.plot.scatter("time", "streamflow")